In [5]:
import os

In [6]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [7]:
DEPTH_CONSTRAINT = 5

In [8]:
from simpletransformers.classification import (
    MultiLabelClassificationModel, MultiLabelClassificationArgs
)
import pandas as pd
import logging
import numpy as np



transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# %%
import dvc.api

df_test = pd.read_json(dvc.api.get_url(
    'datasets/data/query_label/processed/Offshore_Labelled_Query_Classification_Test_V2.json',
    repo='git@github.com:ContextLogic/multitask-llm-rnd.git'
), lines=True)

# %%
df_tax = pd.read_json(dvc.api.get_url(
    'datasets/data/taxonomy/wish_newtax.json',
    repo='git@github.com:ContextLogic/multitask-llm-rnd.git'
), lines=True)
df_tax = df_tax[(df_tax.category_path.apply(len) > 0) & (df_tax.category_path.apply(lambda x: len(x.split(' > ')) <= DEPTH_CONSTRAINT))]

# %%
LABEL_SET = sorted(df_tax.category_path.str.lower().str.strip().tolist()) + ['unknown']

# %%
LABEL_NAME_TO_ID = {i: ind for ind, i in enumerate(LABEL_SET)}
# %%

# %%
def categories2labels(cats):
    if len(cats) == 0:
        cats = ['unknown']
    labs = [0] * len(LABEL_NAME_TO_ID)
    for c in cats:
        labs[LABEL_NAME_TO_ID[c]] = 1
    return labs


In [9]:
model = MultiLabelClassificationModel(
    "xlmroberta",
    "outputs_stage10/best_model",
)

In [10]:
import torch

In [11]:
df_test['labels'] = df_test['query_classification_lists'].apply(
    lambda x: categories2labels([' > '.join(i.lower().strip().split(' > ')[:DEPTH_CONSTRAINT]) for i in x]))
df_test['text'] = df_test['query']

In [12]:
result, model_outputs, wrong_predictions = model.eval_model(
    df_test
)

Running Evaluation:   0%|          | 0/1681 [00:00<?, ?it/s]

In [13]:
result

{'LRAP': 0.5478723087276776, 'eval_loss': 0.0016791872885316423}

In [14]:
from sklearn.metrics import classification_report

In [17]:
print(classification_report(np.array(df_test['labels'].tolist()), model_outputs>.5, zero_division=0,)[-500:])

In [27]:
import pandas as pd

In [28]:
df_qcv2 = pd.read_json(
    "../../../models/multitask_multimodal_multilingual/version_9/clm-epoch=1-step=2600--wish_labelled_query_offshore_test_V2--test.json", 
    lines=True)

In [29]:
df_qcv2 = df_qcv2.sort_values(['batch_indices', 'rank_indices']).groupby('batch_indices').agg({
    'prediction_decoded': lambda x: [i for i in x], 
    'prob': lambda x: [i for i in x]
})

In [30]:
recs = []
for i in df_qcv2.to_dict('records'):
    prediction_decoded_confident = []
    for pred, prob in zip(i['prediction_decoded'], i['prob']):
        if prob >= .05:
            prediction_decoded_confident.append(pred)
    i['prediction_decoded_confident'] = prediction_decoded_confident
    recs.append(i)
df_qcv2 = pd.DataFrame(recs)

In [31]:
df_qcv2['labels'] = df_qcv2['prediction_decoded_confident'].apply(
    lambda x: categories2labels([' > '.join(i.lower().strip().split(' > ')[:DEPTH_CONSTRAINT]) for i in x]))

In [16]:
print(classification_report(np.array(df_test['labels'].tolist()), np.array(df_qcv2['labels'].tolist()))[-500:])